In [21]:
import pandas as pd
import numpy as np
import data_sampling as ds
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn import svm
from sklearn.model_selection import KFold, StratifiedKFold

import time
from sklearn.metrics import confusion_matrix

data_iris = pd.read_csv("iris.csv")
data_glass = pd.read_csv("glass.data")

print(np.shape(data_iris))
print(np.shape(data_glass))
data_glass['target'].values

(150, 5)
(214, 11)


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7])

In [22]:
y = data_glass['target']
data_glass = data_glass.drop(columns = ['target', 'a'])

In [35]:
all_models = [MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(3, 6), random_state=1), 
             Perceptron(tol=1e-3, random_state=0), svm.SVC(gamma='scale', kernel='linear'),
             svm.SVC(gamma='scale', kernel='rbf'), svm.SVC(gamma='scale', kernel='sigmoid')]

In [36]:
kf = KFold(n_splits=5, shuffle=True, random_state=2019)    

In [37]:
#For each model:
for model in all_models:
    print('\n\n======= Model: ', model, '=======')
    # for each fold:
    accuracies = list()
    times = list()
    
    for i, (train_index, test_index) in enumerate(kf.split(data_glass)):
        #Builds the train and validation dataset, according to the current fold:
        y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
        X_train, X_valid = data_glass.iloc[train_index,:].copy(), data_glass.iloc[test_index,:].copy()
        print("Fold", i)
        
        start = time.time()
        model.fit(X_train, y_train)
        end = time.time()
        print("Tempo: ", end - start)
        times.append(end - start)
        
        pred = model.predict(X_valid)
        
        print(confusion_matrix(y_valid, pred, [1,2,3,5,6,7]))
        acc = np.equal(pred, y_valid).astype(float)
        acc = np.mean(acc)
        print('Accuracy:', acc)
        accuracies.append(acc)
    print('Final results: \nMean accuracy:', np.mean(accuracies))
    print('Mean traning model time: ', np.mean(times))



======= Model:  MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(3, 6), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False) =======
Fold 0
Tempo:  0.012248039245605469
[[ 0 16  0  0  0  0]
 [ 0 12  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  5  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  9  0  0  0  0]]
Accuracy: 0.27906976744186046
Fold 1
Tempo:  0.014296770095825195
[[ 0 18  0  0  0  0]
 [ 0 13  0  0  0  0]
 [ 0  4  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  2  0  0  0  0]
 [ 0  6  0  0  0  0]]
Accuracy: 0.3023255813953488
Fold 2
Tempo:  0.015281200408935547
[[ 0 15  0  0  0  0]
 [ 0 13  0  0  0  0]
 [ 0  5  0  0  0  0]
 [ 0  5  0  0  0  0]
 [ 0  1  0  0  0 